In [ ]:
## Watch this to understand how to use the ecosystem daata as quickly as possible
# https: // www.dropbox.com/sh/iysbzchdix2dc55/AAD9t53ntFNtTbyAYyIIdtjRa?dl = 0
# https://www.kaggle.com/c/melbourne-university-seizure-prediction
# Dr Levin Kuhlmann (levin.kuhlmann@monash.edu),

## Imports

In [4]:
import pandas as pd
import scipy.io as sio
import numpy as np
import os
import glob


In [6]:
data_root = '/Users/Shared/Box/Neurovista'
study = 'Pat1Train'
#data_file = 'Pat1Train_1_1.mat'
header = ['ch0',	'ch1',	'ch2',	'ch3',	'ch4',	'ch5',	'ch6',	'ch7',	'ch8',	'ch9',	'ch10',	'ch11',	'ch12',	'ch13',	'ch14',	'ch15']
studyHeader = ['study', 'segment', 'type']
studyHeader_train = ['study','block_id', 'segment', 'type']
analysis_root = '/Users/Shared/Box/Neurovista/analysis'
data_label_root = '/Users/Shared/Box/Neurovista/data_labels'

## Load Train File

In [ ]:
train_labels = pd.read_csv(data_label_root + '/contest_train_data_labels.csv')
train_labels_1 = train_labels[train_labels['class']==1]
#train_labels_1 = train_labels

# new data frame with split value columns
new = train_labels_1["image"].str.split("_", n = 2, expand = True)
  
# making separate first name column from new data frame
train_labels_1["patient"]= new[0]
  
# making separate last name column from new data frame
train_labels_1["segment"]= new[1]

train_labels_1["segment"] = train_labels_1["segment"].astype(int)

train_labels_1["block_id"]= np.ceil(np.array(train_labels_1['segment']/6))
train_labels_1["block_id"] = train_labels_1["block_id"].astype(int)

train_labels_1 = train_labels_1[train_labels_1['patient']==study]
train_labels_1["class"] = train_labels_1["class"].astype(int)

train_labels_1 = train_labels_1[train_labels_1['segment']<=150]

## Load Test files

In [ ]:
test_labels = pd.read_csv(data_label_root +'/contest_test_data_labels_public.csv')
test_labels_public = test_labels[test_labels['usage']=='Public']
# new data frame with split value columns
new = test_labels_public["image"].str.split("_", n = 2, expand = True)
  
# making separate first name column from new data frame
test_labels_public["patient"]= new[0]
  
# making separate last name column from new data frame
test_labels_public["segment"]= new[1]

test_labels_public["class"] = test_labels_public["class"].astype(int)
test_labels_public = test_labels_public[test_labels_public['patient']==study]

test_labels_public

In [ ]:
Pat1Test_df = pd.DataFrame()
matFiles = data_root+study+'/'+test_labels_public["image"]+'.mat'
for matFilePath in matFiles:
  print(matFilePath)
  loadedMatFile = sio.loadmat(matFilePath)['data']

  file_name = matFilePath.split('/')[-1].split('.')[0]
  file_name_row = test_labels_public[test_labels_public['image'] == file_name]
  newColumns = [file_name_row[['patient','segment','class']].values] * np.shape(loadedMatFile)[0]
  newColumns = np.squeeze(newColumns,axis=1)

  study_df = pd.DataFrame(newColumns, columns=studyHeader)
  mat_df = pd.DataFrame(loadedMatFile, columns=header)
  result = pd.concat([study_df, mat_df], axis=1)
  Pat1Test_df = pd.concat([Pat1Test_df,result])

Pat1Test_df.study = Pat1Test_df.study.astype(str)
Pat1Test_df.segment = Pat1Test_df.segment.astype(int)
Pat1Test_df['type'] = Pat1Test_df['type'].astype(int)